# 1. NLP Exploration 2: Webscraping 
This exploration is an extension of **Domains Classifier V4**. We looked at just the domain name in that example, and utilized segmentation and a few other things in order to create a **vocabulary** of words that occur in the domain name. This vocabulary then one hot encoded in order to represent inputs, and then these inputs were fed into a **random forest** in order to try and map to the correct output class. 

We are now going to try and extended that by making use of **Webscraping**. Working with the same data set we will scrape the associated webpages and try to gather as much information as we can that may help us make predictions. Some things to note:
1. We will start by just looking at the home page. This clearly does not give us 100% of the information we may need, but it will be the first iteration.
2. We will also only be looking at visible text and not at css class names, commented out code, or anything else that would not be visible to a user. 
2. Based on the number of domains we have scores for (2000) we will need to be wary of how large we let our vocabulary size grow to. 

With that said, the first step is for us to actually scrape the home page of each domain. We can start by importing our necessary modules and loading in our data. 

In [69]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from util import (preprocess_analyst_scored_domains, class_imbalance)
%matplotlib inline

# Seaborn Plot Styling
sns.set(style="white", palette="husl")
sns.set_context("poster")
sns.set_style("ticks")

In [70]:
# Load our data
df = pd.read_csv('PPSDomains.csv')
df.head()

,domain,analystResult,BellonaStatus,medianLoadTime,speedPercentile,trafficDataRank,trafficDataReachRank,reachPerMillionValue,pageViewsPerMillionValue,pageViewsRankValue,usageStatisticRankValue,reachRankValue,historyDataScore,domainAge,registrantContactCountry,privateRegistrationStatus
0,07449m.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,68.0,UNITED STATES,"{""registrantName"":""Domain Administrator"",""simi..."
1,1037kissfm.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,6955.0,UNITED STATES,NaN
2,1041kqth.com,NaN,TODO,7687.0,1.0,1941173.0,1872730.0,0.2,0.01,2322074.0,2025813.0,2060156.0,3,1251.0,UNITED STATES,NaN
3,1049maxcountry.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,978.0,UNITED STATES,NaN
4,1053thebuzz.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,6221.0,UNITED STATES,NaN


In [71]:
# Call preprocess Utility Function
analystScoredDomains = preprocess_analyst_scored_domains(df)
analystScoredDomains.head()

,domain,analystResult,domainAge,registrantContactCountry,privateRegistrationStatus
10,10thousandcouples.com,0,-0.298104,0,0
11,1170kfaq.com,0,0.575157,1,0
12,11alive.com,0,1.205884,1,0
13,120gdyiyuan.com,0,-1.361280,1,0
14,12news.com,0,1.117373,1,0


---

<br>
## 1.1 Let the webscraping begin
Okay now that our data has been loaded and preprocessed, we can begin the process of webscrapping. In order to do that we will make use of the `urllib` and `BeatifulSoup` libraries.

In [72]:
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.request

Now, lets go through the process of extracting the home page content for one domain, and then create functions to apply that to all of our domains. 

In [73]:
# Function to take HTML and return only the visible text
def text_from_html(body):
  soup = BeautifulSoup(body, 'html.parser')
  texts = soup.findAll(text=True)
  visible_texts = filter(tag_visible, texts)
  return u" ".join(t.strip() for t in visible_texts)

# Function that determines whether the text would be visible on the web page
def tag_visible(element):
  if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
      return False
  if isinstance(element, Comment):
      return False
  return True

In [74]:
# Request the particular webpage
html = urllib.request.urlopen('http://12news.com').read()

# Get the page text from the html
page_text = text_from_html(html)

In order to get the text in the format we want, we will utilize the `nltk` library.

In [75]:
import nltk

# Tokenize all of the page text
page_text_tokens = nltk.word_tokenize(page_text)

In [96]:
# Create new column for page content, set to an empty string
analystScoredDomains['page_content'] = ''

# Get index of our test example
index = analystScoredDomains[analystScoredDomains['domain'] == '12news.com'].index[0]

# Using .at accessor, set value of page_content column to be equal to page_text_tokens
analystScoredDomains.at[index, 'page_content'] = page_text_tokens

# Display first ten values, and then the total length of the array
display(analystScoredDomains.at[index, 'page_content'][:10])
display(len(analystScoredDomains.at[index, 'page_content']))

['WATCH', 'LIVE', 'On', 'Air', '3:55PM', '80', 'Phoenix', ',', 'AZ', 'Phoenix']

1094

Awesome, so we did it for one entry, let's create a function that does this process for us. We also are going to want to ensure that we peform the necessary text processing. To do that we will create a tokenizer function. 

In [97]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
stopwords = set(w.rstrip() for w in open('stopwords.txt'))         # grab stop words 

# Create Custom tokenizer function 
def custom_tokenizer(s):
  s = s.lower()
  tokens = nltk.tokenize.word_tokenize(s)                        # essentially string.split()
  tokens = [t for t in tokens if len(t) > 2]                     # get rid of short words
  tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens]     # get words to base form
  tokens = [t for t in tokens if t not in stopwords]
  return tokens

Great! We now have all of the functions we need, it is time to make all of our requests, scrape the pages, and set the values for `page_content` in our dataframe. We also want to index each word!

In [ ]:
word_index_map = {}           # Vocabulary
current_index = 0

for index, row in analystScoredDomains.iterrows():
  
  try: 
    # Create full URL to make valid request
    domain = analystScoredDomains.at[index, 'domain']
    full_domain_request = 'http://' + domain

    # Request the particular webpage
    html = urllib.request.urlopen(full_domain_request).read()

    # Get the page text from the html
    page_text = text_from_html(html)

    # Tokenize all of the page text
    page_text_tokens = nltk.word_tokenize(page_text)
    
    # Using .at accessor, set value of page_content column to be equal to page_text_tokens
    analystScoredDomains.at[index, 'page_content'] = page_text_tokens
    
#   except urllib.error.HTTPError as err: 
#     print('The request could not go through: ', )
    
#   except urllib.error.URLError as err: 
#     print('The request could not go through: ', )

  except: 
    print("error occured")